In [46]:
import json
import requests
import praw
import numpy

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 5
wanted_count_sub = 5
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)
used_authors = []
used_subreddits = []
#offset_limit = 4
    
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in unwanted_users:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit):
    #if subreddit in used_subreddits:
    #    return []
    #else:
    url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count)
    response = requests.get(url).json()
    fetched_authors = getKeysFromDict(response['aggs']['author'])
    return fetched_authors

def getMostActiveSubRedditForAuthor(author):
    #if author in used_authors:
    #    return []
    #else:
    url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub)
    response = requests.get(url).json()
    fetched_subreddits = getKeysFromDict(response['aggs']['subreddit'])
    for sub in used_subreddits:
            if (sub not in used_subreddits):
                used_subreddits.append(sub)
    return fetched_subreddits

In [47]:
%%time
relation_array = []
used_subreddits = []
used_authors = []

# Begin data fetching
# Get data from most active users in specified subreddit
users_1 = getMostActiveAuthorsInSubReddit("The_Donald")
print("Fetched users lvl 1: ", "The_Donald")
print("users in The_Donald", users_1)

# Append specified subreddit to used_subreddit list
used_subreddits.append("The_Donald")

# Begin scraping specified subreddit (Level 1)

for user in users_1:
    print("Entering user Lvl 1:", user)
    subs = getMostActiveSubRedditForAuthor(user)
    print("-- Fetched subs lvl 2 for user", user, ": ", subs)
    for sub2 in subs:
        # Make this global
        if sub2 != "The_Donald":
            relation_array.append(["The_Donald", sub2])
        if sub2 not in used_subreddits:
            used_subreddits.append(sub2)
            print("-- Entering sub Lvl 2:", sub2)
        # Retrieve users from sub
            users_2 = getMostActiveAuthorsInSubReddit(sub2)
            print("-- users2 in ", sub2, "is ", users_2)
            for user2 in users_2:
                if user2 not in used_authors:
                    print("-- -- Entering user Lvl 2:", user2)
                    subs3 = getMostActiveSubRedditForAuthor(user2)
                    print("-- -- Fetched subs lvl 3 for user ", user2, ": ", subs3)
                    used_authors.append(user2)
                    for sub3 in subs3:
                        if sub3 != sub2:
                            relation_array.append([sub2, sub3])
                        if sub3 not in used_subreddits:
                            used_subreddits.append(sub3)
                            print("-- -- Entering sub Lvl 3:", sub3)

Fetched users lvl 1:  The_Donald
users in The_Donald ['mangoroti', 'PropellerPowerCurve', 'ENVYNITAZ', 'thenew23rd', 'DragonsAndDiamonds']
Entering user Lvl 1: mangoroti
-- Fetched subs lvl 2 for user mangoroti :  ['The_Donald', 'florida', 'RightwingLGBT', 'politics', 'SandersForPresident']
-- Entering sub Lvl 2: florida
-- users2 in  florida is  ['ExCap2', 'Burner_Acount', 'says_harsh_things', 'ICanQuitWhenverIWant', 'Automatic-Mention', 'marinersalbatross']
-- -- Entering user Lvl 2: ExCap2
-- -- Fetched subs lvl 3 for user  ExCap2 :  ['florida', 'Unemployment', 'personalfinance', 'politics', 'IRS']
-- -- Entering sub Lvl 3: Unemployment
-- -- Entering sub Lvl 3: personalfinance
-- -- Entering sub Lvl 3: politics
-- -- Entering sub Lvl 3: IRS
-- -- Entering user Lvl 2: Burner_Acount
-- -- Fetched subs lvl 3 for user  Burner_Acount :  ['florida', 'dataisbeautiful', 'PLC', 'arduino', 'Coronavirus']
-- -- Entering sub Lvl 3: dataisbeautiful
-- -- Entering sub Lvl 3: PLC
-- -- Entering s

In [48]:
relation_array

[['The_Donald', 'florida'],
 ['florida', 'Unemployment'],
 ['florida', 'personalfinance'],
 ['florida', 'politics'],
 ['florida', 'IRS'],
 ['florida', 'dataisbeautiful'],
 ['florida', 'PLC'],
 ['florida', 'arduino'],
 ['florida', 'Coronavirus'],
 ['florida', 'PurplePillDebate'],
 ['florida', 'news'],
 ['florida', 'television'],
 ['florida', 'funny'],
 ['florida', 'worldnews'],
 ['florida', 'politics'],
 ['florida', 'wallstreetbets'],
 ['florida', 'news'],
 ['florida', 'tampa'],
 ['florida', 'USF'],
 ['florida', 'orlando'],
 ['florida', 'keto'],
 ['florida', 'worldnews'],
 ['florida', 'news'],
 ['florida', 'todayilearned'],
 ['florida', 'FloridaCoronavirus'],
 ['The_Donald', 'RightwingLGBT'],
 ['RightwingLGBT', 'gaybrosgonemild'],
 ['RightwingLGBT', 'conservatives'],
 ['RightwingLGBT', 'news'],
 ['RightwingLGBT', 'TumblrInAction'],
 ['RightwingLGBT', 'progun'],
 ['RightwingLGBT', 'Eve'],
 ['RightwingLGBT', 'gunpolitics'],
 ['RightwingLGBT', 'LGBDropTheT'],
 ['RightwingLGBT', 'BadDragon'

In [1]:
relation_array = []
users_1 = getMostActiveAuthorsInSubReddit('The_Donald')
used_subreddits.append("The_Donald")

subcount = {}
for user in users_1:
    subs = getMostActiveSubRedditForAuthor(user)
    for sub in subs:
        #if sub not in list(subcount.keys()):
        #    subcount[sub] = 1
        #else:
        #   subcount[sub] += 1
        relation_array.append(['The_Donald', sub])
        # NIVÅ 2 START
        users_2= getMostActiveAuthorsInSubReddit(sub)
        for user2 in users_2:
            subs2 = getMostActiveSubRedditForAuthor(user)
            for sub2 in subs2:
                #if sub2 not in list(subcount.keys()):
                #    subcount[sub2] = 1
                #else:
                #    subcount[sub2] += 1
                if user2 not in used_authors:
                    relation_array.append([sub, sub2])
                    used_subreddits.append(sub)
            used_authors.append(user2)
        # NIVÅ 2 SLUTT
        
        
        
        
        
        
    used_authors.append(user)


relation_array

IndentationError: unexpected indent (<ipython-input-1-d9a47d917cdf>, line 16)

In [3]:
print(subcount)

{'The_Donald': 5, 'florida': 4}


### 